In [1]:
import pandas as pd
import os
cwd = os.getcwd()
year = 5

In [25]:
df1 = pd.read_excel(cwd + '/Retail Turnover State by Industry Subgroup Original.xlsx', sheet_name='Data1')
df2 = pd.read_excel(cwd + '/Formula Sheet.xlsx', sheet_name='Industry Manager Title')
df3 = pd.read_excel(cwd + '/Formula Sheet.xlsx', sheet_name='Manager Annual Wage')

df1.drop(columns=['Unnamed: 0'], inplace=True)
df1 = (
    df1.tail(year * 12 + 1)
        .pct_change(fill_method=None)
        .dropna(how='all')
        .mean()
        .to_frame(name='Turnover Rate')  # Name the value column
        .reset_index()     # Name the former index as a column
)

# Now split the 'index' column into parts
df1[['Part1', 'State', 'Industry','Part2']] = df1['index'].str.split(';', expand=True)

# Clean spaces
df1['State'] = df1['State'].str.strip()
df1['Industry'] = df1['Industry'].str.strip()

# Drop the useless columns
df1 = df1[['State', 'Industry', 'Turnover Rate']]
df1 = df1.merge(df2, how='left', left_on='Industry', right_on='Retail Trade')
df3 = df3.rename(columns={'State/Territory': 'State'}).melt(id_vars='State', var_name='Title', value_name='Salary')
df1 = df1.merge(df3, how='left', on=['State', 'Title'])

In [30]:

with pd.ExcelWriter(cwd + '/Formula Sheet.xlsx', mode='a', engine='openpyxl') as writer:
    if 'Retail Industry Metrics' in writer.book.sheetnames:
        del writer.book['Retail Industry Metrics']
    df1.to_excel(writer, sheet_name='Retail Industry Metrics', index=False)

In [47]:
sampleData = pd.read_excel(cwd + '/Formula Sheet.xlsx', sheet_name='Business Data')
sampleData.dropna(inplace=True)
sampleData

,ID,BusinessName,Suburb,State,Postcode,Industry,AskingPrice,StatedSAV,Revenue,Outgoings,...,SAVIncluded,RentGrowthIncluded,RentGrowth,BusinessType,SGA,TradingDays,ListingStatus,SoldPrice,URL,DateAdded
0,20250001,Gotcha Fresh Tea,Eastern Creeks,New South Wales,2766.0,"Cafes, restaurants and takeaway food services",240000.0,0.0,547752.0,231960.0,...,0.0,1.0,0.04,Small Business,0.0,7.0,1.0,0.0,https://www.seekbusiness.com.au/business-listi...,2025-01-05
1,20250002,Deluca Coffee,Bondi,New South Wales,2026.0,"Cafes, restaurants and takeaway food services",75000.0,0.0,243000.0,49800.0,...,0.0,0.0,0.00,Small Business,0.0,7.0,1.0,0.0,https://www.seekbusiness.com.au/business-listi...,2025-01-06
2,20250003,Restaurant West,Harris Park,New South Wales,2150.0,"Cafes, restaurants and catering services",300000.0,0.0,936000.0,412412.0,...,1.0,0.0,0.00,Medium Business,0.0,7.0,1.0,0.0,https://www.seekbusiness.com.au/business-listi...,2025-01-07
3,20250004,Bakery Cafe Inner West,Balmain,New South Wales,2041.0,"Cafes, restaurants and catering services",100000.0,0.0,228000.0,90000.0,...,0.0,0.0,0.00,Small Business,0.0,7.0,1.0,0.0,https://www.seekbusiness.com.au/business-listi...,2025-01-08
4,20250005,Espresso Bar Inner West,Haberfield,New South Wales,2045.0,"Cafes, restaurants and catering services",170000.0,0.0,468000.0,137696.0,...,0.0,0.0,0.00,Small Business,0.0,7.0,1.0,0.0,https://www.seekbusiness.com.au/business-listi...,2025-01-09
5,20250006,Cafe Northern Beaches,Dee Why,New South Wales,2099.0,"Cafes, restaurants and catering services",184000.0,0.0,624000.0,185296.0,...,1.0,0.0,0.00,Small Business,0.0,7.0,1.0,0.0,https://www.seekbusiness.com.au/business-listi...,2025-01-10
6,20250007,Thai Restaurant,Kellyville,New South Wales,2155.0,"Cafes, restaurants and catering services",870000.0,0.0,1820000.0,624000.0,...,1.0,0.0,0.00,Medium Business,0.0,7.0,1.0,0.0,https://www.seekbusiness.com.au/business-listi...,2025-01-11


In [48]:
sampleData.to_json('../frontend/src/data/businessData.json', orient='records', lines=False)

In [49]:
df1.to_json('../frontend/src/data/retailIndustryMetrics.json', orient='records', lines=False)